## Semantic Kernel - Chat On your SQL Data

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.23.0"
#r "nuget: System.Data.SqlClient, 4.8.3"
#r "nuget: Microsoft.Extensions.Configuration, 6.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 6.0.0"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;
using System.ComponentModel;
using System.Text.Json.Serialization;
using System.Data.SqlClient;
using Microsoft.Extensions.Configuration;


Installed Packages Microsoft.Extensions.Configuration, 6.0.0 Microsoft.Extensions.Configuration.Json, 6.0.0 Microsoft.SemanticKernel, 1.23.0 System.Data.SqlClient, 4.8.3

In [2]:
var config = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("config/appSettings.json")
    .Build();

var username = config["SQL:Username"];
var password = config["SQL:Password"];
var server = config["SQL:Server"];
var database = config["SQL:Database"];

Console.WriteLine((username))

sqladmin


## Class to Retrieve DateTime with Semantic Kernel

In [3]:
public class TimeInformation
    {
        [KernelFunction]
        [Description("Retrieves the current time in UTC.")]
        public string GetCurrentUtcTime() => DateTime.UtcNow.ToString("R");
    }

In [4]:
using System;
using System.ComponentModel;
using System.Data.SqlClient;
using System.Text;
using Microsoft.SemanticKernel;

public class SQLPlugIn
{
    private readonly string _connectionString;

    public SQLPlugIn()
    {
        var config = new ConfigurationBuilder()
        .SetBasePath(Directory.GetCurrentDirectory())
        .AddJsonFile("config/appsettings.json")
        .Build();

        var username = config["SQL:Username"];
        var password = config["SQL:Password"];
        var server = config["SQL:Server"];
        var database = config["SQL:Database"];

        _connectionString = $"Server=tcp:{server};Initial Catalog={database};Persist Security Info=False;User ID=sqladmin;Password={password};MultipleActiveResultSets=False;Encrypt=True;TrustServerCertificate=False;Connection Timeout=30;";
    }

    [KernelFunction, Description("Get list of tables in the database")]
    public string GetTables()
    {
        var query = "SELECT TABLE_SCHEMA, TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'";
        return ExecuteQuery(query);
    }

    [KernelFunction, Description("Get schema information for all tables including column details")]
    public string GetTableSchema()
    {
        var query = "SELECT TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, DATA_TYPE, IS_NULLABLE, CHARACTER_MAXIMUM_LENGTH FROM INFORMATION_SCHEMA.COLUMNS ORDER BY TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION";
        return ExecuteSchemaQuery(query);
    }
    private string ExecuteQuery(string query)
    {
        var sb = new StringBuilder();
        sb.Append("<table><tr><th>Schema</th><th>Table</th></tr>");

        using (var connection = new SqlConnection(_connectionString))
        using (var command = new SqlCommand(query, connection))
        {
            connection.Open();
            using (var reader = command.ExecuteReader())
            {
                while (reader.Read())
                {
                    sb.Append($"<tr><td>{reader["TABLE_SCHEMA"]}</td><td>{reader["TABLE_NAME"]}</td></tr>");
                }
            }
        }

        sb.Append("</table>");
        return sb.ToString();
    }
\n    private string ExecuteSchemaQuery(string query)\n    {\n        var sb = new StringBuilder();\n        sb.Append(\"<table><tr><th>Schema</th><th>Table</th><th>Column</th><th>Data Type</th><th>Nullable</th><th>Max Length</th></tr>\");\n\n        using (var connection = new SqlConnection(_connectionString))\n        using (var command = new SqlCommand(query, connection))\n        {\n            connection.Open();\n            using (var reader = command.ExecuteReader())\n            {\n                while (reader.Read())\n                {\n                    var maxLength = reader[\"CHARACTER_MAXIMUM_LENGTH\"] == DBNull.Value ? \"N/A\" : reader[\"CHARACTER_MAXIMUM_LENGTH\"].ToString();\n                    sb.Append($\"<tr><td>{reader[\"TABLE_SCHEMA\"]}</td><td>{reader[\"TABLE_NAME\"]}</td><td>{reader[\"COLUMN_NAME\"]}</td><td>{reader[\"DATA_TYPE\"]}</td><td>{reader[\"IS_NULLABLE\"]}</td><td>{maxLength}</td></tr>\");\n                }\n            }\n        }\n\n        sb.Append(\"</table>\");\n        return sb.ToString();\n    }\n}


In [6]:
//Create Kernel builder
var builder = Kernel.CreateBuilder();
// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
builder.Plugins.AddFromType<TimeInformation>();
builder.Plugins.AddFromType<SQLPlugIn>();
var kernel = builder.Build();


#pragma warning disable SKEXP0001

OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() 
{
    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
};

In [7]:
var ask = "What is the current time in UTC?";
var result = await kernel.InvokePromptAsync(ask, new(openAIPromptExecutionSettings));

Console.WriteLine(result);

The current time in UTC is Friday, 20 June 2025, 14:14:04.


In [8]:
var ask = "What are the tables in my database?";
var result = await kernel.InvokePromptAsync(ask, new(openAIPromptExecutionSettings));

Console.WriteLine(result);

Your database contains the following tables:

1. Capabilities
2. Incidents
3. Units
4. UnitCapabilities
5. IncidentNotes

If you need details about the columns or data in any of these tables, let me know!


In [ ]:
var ask = \"What is the schema of the tables in my database?\";\nvar result = await kernel.InvokePromptAsync(ask, new(openAIPromptExecutionSettings));\n\nConsole.WriteLine(result);